In [ ]:
import requests
from IPython.display import Markdown, display

def get_citation_from_doi(doi, format_type='markdown'):
    """
    Fetch DOI metadata and return citation in format:
    "AuthorLastName. et al. Journal Year, Volume (Issue), Pages"
    with year in bold, "et al." and journal name in italic
    
    Args:
        doi: DOI string (e.g., "10.1063/5.0180431")
        format_type: 'markdown' (default) or 'html' or 'plain'
    
    Returns:
        Formatted citation string
    """
    # Clean the DOI (remove any https://doi.org/ prefix if present)
    if doi.startswith('http'):
        doi = doi.split('doi.org/')[-1]
    
    # Fetch metadata from CrossRef API
    url = f"https://api.crossref.org/works/{doi}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()['message']
        
        # Extract author information
        authors = data.get('author', [])
        if authors:
            first_author = authors[0].get('family', 'Unknown')
            if len(authors) > 1:
                if format_type == 'markdown':
                    author_str = f"{first_author}. *et al.*"
                elif format_type == 'html':
                    author_str = f"{first_author}. <i>et al.</i>"
                else:
                    author_str = f"{first_author}. et al."
            else:
                author_str = f"{first_author}."
        else:
            author_str = "Unknown."
        
        # Extract journal name (abbreviated if available, otherwise full)
        journal = data.get('short-container-title', data.get('container-title', ['Unknown Journal']))[0]
        
        # Format journal name in italic
        if format_type == 'markdown':
            journal = f"*{journal}*"
        elif format_type == 'html':
            journal = f"<i>{journal}</i>"
        
        # Extract year
        pub_date = data.get('published-print', data.get('published-online', {}))
        year = pub_date.get('date-parts', [[None]])[0][0] if pub_date else 'N/A'
        
        # Format year in bold
        if format_type == 'markdown':
            year = f"**{year}**"
        elif format_type == 'html':
            year = f"<b>{year}</b>"
        
        # Extract volume
        volume = data.get('volume', '')
        
        # Extract issue
        issue = data.get('issue', '')
        
        # Extract pages
        page = data.get('page', '')
        
        # Format citation
        citation = f"{author_str} {journal} {year}"
        
        if volume:
            citation += f", {volume}"
        
        if issue:
            citation += f" ({issue})"
        
        if page:
            citation += f", {page}"
        
        return citation
        
    except requests.exceptions.RequestException as e:
        return f"Error fetching DOI: {e}"
    except (KeyError, IndexError) as e:
        return f"Error parsing metadata: {e}"

def display_citation(doi):
    """Display citation with proper markdown formatting in Jupyter"""
    citation = get_citation_from_doi(doi, format_type='markdown')
    display(Markdown(citation))


In [3]:
# Example usage
example_doi = "10.1038/s41467-023-36329-y"

# Display with markdown formatting (renders in notebook)
print("Formatted citation:")
display_citation(example_doi)

# Or get as plain text
print("\nAs markdown text:")
print(get_citation_from_doi(example_doi, format_type='markdown'))

# Or get as HTML
print("\nAs HTML:")
print(get_citation_from_doi(example_doi, format_type='html'))

# Or get as plain text
print("\nAs plain text:")
print(get_citation_from_doi(example_doi, format_type='plain'))

Formatted citation:


Musaelian. *et al.* *Nat Commun* **2023**, 14 (1)


As markdown text:
Musaelian. *et al.* *Nat Commun* **2023**, 14 (1)

As HTML:
Musaelian. <i>et al.</i> <i>Nat Commun</i> <b>2023</b>, 14 (1)

As plain text:
Musaelian. et al. Nat Commun 2023, 14 (1)
